In [1]:
!pip uninstall transformers torch accelerate -y
!pip install transformers[torch] accelerate

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: torch 2.3.0+cu121
Uninstalling torch-2.3.0+cu121:
  Successfully uninstalled torch-2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [2]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=4007ce35a1aea8e10d8cf7811017c66ca6149db03ea343dc79a7edeafc8b367f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [4]:
!pip install datasets

In [5]:
from huggingface_hub import notebook_login

notebook_login()

datos_wav --> todos los datos

texto_final --> sololos que tiene el modelo de audio

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from datasets import load_dataset,Audio
dataset = load_dataset("csv",data_files="/content/drive/MyDrive/datos_wav.csv",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
dataset = dataset.remove_columns(["Unnamed: 0", "ID_frase", "conv","speaker","video_name","emotion_x"])
dataset=dataset.rename_column('emocion_code','label')

In [9]:
train_testvalid = dataset.train_test_split(test_size=0.2, seed=1)
test = train_testvalid['test']

train_valid = train_testvalid['train'].train_test_split(test_size=0.25, seed=1)
train = train_valid['train']
val = train_valid['test']


In [10]:
train

Dataset({
    features: ['text', 'label'],
    num_rows: 5876
})

In [11]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [14]:
token_test = test.map(preprocess_function, batched=True)
token_train= train.map(preprocess_function, batched=True)
token_val= val.map(preprocess_function, batched=True)

Map:   0%|          | 0/1959 [00:00<?, ? examples/s]

Map:   0%|          | 0/5876 [00:00<?, ? examples/s]

Map:   0%|          | 0/1959 [00:00<?, ? examples/s]

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

In [17]:
id2label = {0: "ANGER", 1: "DISGUST",2:"FEAR",3:"JOY",4:"SADNESS",5:"SURPRISE"}
label2id = {"ANGER": 0, "DISGUST": 1,"FEAR":2,"JOY":3,"SADNESS":4,"SURPRISE":5}

In [18]:
from transformers import DebertaForSequenceClassification, TrainingArguments, Trainer

model = DebertaForSequenceClassification.from_pretrained(
    "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", num_labels=6,id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

You are using a model of type roberta to instantiate a model of type deberta. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.in_proj.weight', 'encoder.layer.0.attention.self.q_bias', 'encoder.layer.0.attention.self.v_bias', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encode

In [ ]:
training_args = TrainingArguments(
    output_dir="distilroberta_finanzas_normal",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_train,
    eval_dataset=token_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.541508,0.459929
2,No log,1.623382,0.467075
3,No log,1.667610,0.454824
4,No log,1.626917,0.475753
5,No log,1.673933,0.474222
6,No log,1.764293,0.473711
7,No log,1.743101,0.482899
8,No log,1.787392,0.485452
9,No log,1.836678,0.476263
10,No log,1.882410,0.477284


In [ ]:
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoModelForSequenceClassification, pipeline

# data = load_dataset('csv',data_files="datos_wav.csv", split="train")
# data = data.remove_columns(["Unnamed: 0","ID_frase", "conv","speaker","video_name","emotion_x"])
# data=data.rename_column('emocion_code','label')

# val=load_dataset('csv',data_files="/content/val_disbert (1).csv", split="train")

task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline="Drigoro/distilroberta_finanzas_normal",
    data=val,
    metric=evaluate.combine(["accuracy"]),
    label_mapping={"ANGER": 0, "DISGUST": 1,"FEAR":2,"JOY":3,"SADNESS":4,"SURPRISE":5}
)
print(eval_results)